## Llama 3.3 70B Instruct - SAE Inference

The following are multiple examples of getting activations and searching Llama 3.3 70B on Neuronpedia using [Goodfire's Layer 50 SAE](https://huggingface.co/Goodfire/Llama-3.3-70B-Instruct-SAE-l50/tree/main).

In the following examples, we focus on using feature 7336 from the layer 50 SAE, which is a feature about rice.

_Dashboards are not yet uploaded for this SAE, so the links to Neuronpedia dashboards will not work._

### Limitations

- Non-batch requests: Limit of 1024 tokens per prompt
- Batch requests: Limit of 256 tokens per prompt, with a max of 4 prompts per batch request
- Steering is not yet enabled - we expect this to be available soon - if you need this urgently, contact us at team@neuronpedia.org.


In [1]:
# ===== Constants =====

# We specify the model and SAE source ID
model_id = "llama3.3-70b-it"
source = "50-resid-post-gf"

# We'll use a "Rice" feature as an example
index = 7336

# Example single prompt
single_prompt = "I like to eat rice."

# Example multiple (batch) prompts
batch_prompts = [
    "I like to eat rice.",
    "I like to eat pizza.",
    "I like to eat sushi.",
    "I like to pet dogs.",
]

# ===== Imports and Config =====

%pip install neuronpedia

from dotenv import load_dotenv
from neuronpedia.requests.activation_request import ActivationRequest
from neuronpedia.requests.source_activation_request import SourceActivationRequest
from neuronpedia.requests.topk_by_token_request import TopKByTokenRequest

# Load NEURONPEDIA_API_KEY from .env file
load_dotenv()

### [less safe] Set Neuronpedia API key manually (get your key from neuronpedia.org/account)
# import os
# os.environ["NEURONPEDIA_API_KEY"] = "YOUR_KEY_HERE"


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [ ]:
# Example 1: Activations for Single Feature, Single Prompt
# Maximum of 1024 tokens per prompt

activation = ActivationRequest().compute_activation_for_text(
    model_id, source, index, single_prompt
)

print(activation)

In [ ]:
# Example 2: Activations for Single Feature, Multiple Prompts (Batch)
# Maximum of 256 tokens per prompt, up to 4 prompts per request

activations = ActivationRequest().compute_activation_for_texts(
    model_id, source, index, batch_prompts
)

for activation in activations:
    print(activation)


In [ ]:
# Example 3: Top features by activations in SAE, Single Prompt
# Maximum of 1024 tokens per prompt

# We sort the results by highest activation at "rice", the token at index 5.
# We can also exclude that argument if we want to see the top features for all tokens.
activations = SourceActivationRequest().compute_top_features_for_text(
    model_id,
    source,
    single_prompt,
    ignore_bos=True,
    sort_results_by_activations_at_token_indexes=[5],
    # see method for more options
)
for activation in activations:
    print("")
    print(activation)
    print(
        f"https://neuronpedia.org/{activation.modelId}/{activation.source}/{activation.index}"
    )
    print("-" * 50)

In [ ]:
# Example 4: Top features by activations in SAE, Multiple Prompts (batch)
# Maximum of 256 tokens per prompt, up to 4 prompts per request

activations_batch = SourceActivationRequest().compute_top_features_for_texts(
    model_id,
    source,
    batch_prompts,
    ignore_bos=True,
    # see method for more options
)

for i, activations in enumerate(activations_batch):
    print("")
    print(f"Prompt: {batch_prompts[i]}")
    print("=" * 50)
    for activation in activations:
        print("")
        print(activation)
        print(
            f"https://neuronpedia.org/{activation.modelId}/{activation.source}/{activation.index}"
        )
        print("-" * 50)

In [ ]:
# Example 5: TopK by Token for All Features in SAE, Single Prompt
# Maximum of 1024 tokens per prompt

topk_by_token = TopKByTokenRequest().compute_top_features_by_token_for_text(
    model_id,
    source,
    single_prompt,
    num_results=5,
    ignore_bos=True,
)

for result in topk_by_token:
    print(f"\nToken: '{result.token}' at position {result.position}")
    print(f"Top {len(result.topFeatures)} features:")
    for i, feature_activation in enumerate(result.topFeatures, 1):
        print(
            f"  {i}. Feature {feature_activation.feature.index}: {feature_activation.activationValue:.4f}"
        )
        print(
            f"     https://neuronpedia.org/{feature_activation.feature.model}/{feature_activation.feature.source}/{feature_activation.feature.index}"
        )

In [ ]:
# Example 6: TopK by Token for All Features in SAE, Multiple Prompts (batch)
# Maximum of 256 tokens per prompt, up to 4 prompts per request

topk_by_token_batch = TopKByTokenRequest().compute_top_features_by_token_for_texts(
    model_id,
    source,
    batch_prompts,
    num_results=5,
    ignore_bos=True,
)

for batch_idx, topk_by_token in enumerate(topk_by_token_batch):
    print(f"\nPrompt {batch_idx + 1}: {batch_prompts[batch_idx]}")
    print("=" * 50)
    for result in topk_by_token:
        print(f"\nToken: '{result.token}' at position {result.position}")
        print(f"Top {len(result.topFeatures)} features:")
        for i, feature_activation in enumerate(result.topFeatures, 1):
            print(
                f"  {i}. Feature {feature_activation.feature.index}: {feature_activation.activationValue:.4f}"
            )
            print(
                f"     https://neuronpedia.org/{feature_activation.feature.model}/{feature_activation.feature.source}/{feature_activation.feature.index}"
            )

In [2]:
# Example 7: All activations for entire SAE, Single or Multiple Prompts
# Maximum of 1024 tokens per prompt if single prompt, or 256 tokens per prompt if multiple prompts
# Maximum of 4 prompts per request

# Returns prompts x tokens x activations. Does not return dashboards or explanations.

results = ActivationRequest().compute_all_source_activations_for_text(
    model_id, source, batch_prompts
)

for result in results:
    print(result["tokens"])
    print(
        f"{len(result['result'])} feature results. Parse these activations to find active features."
    )


Sending POST request to http://localhost:3000/api/activation/source
Got a successful response.
['<|begin_of_text|>', 'I', ' like', ' to', ' eat', ' rice', '.']
65536 feature results. Parse these activations to find active features.
['<|begin_of_text|>', 'I', ' like', ' to', ' eat', ' pizza', '.']
65536 feature results. Parse these activations to find active features.
['<|begin_of_text|>', 'I', ' like', ' to', ' eat', ' sushi', '.']
65536 feature results. Parse these activations to find active features.
['<|begin_of_text|>', 'I', ' like', ' to', ' pet', ' dogs', '.']
65536 feature results. Parse these activations to find active features.
